In [1]:
import os
import glob
from sklearn.model_selection import train_test_split

/home/chlee/.conda/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-17 08:21:55.628855: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-17 08:21:55.631557: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-17 08:21:55.676286: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appro

#### Dataset (ldm ver)

In [2]:
# 경로 설정
slit_dir = "/camin1/chlee/Data/cornea_image/segmented/Lt/slit/whole"
slit_beam_dir = "/camin1/chlee/Data/cornea_image/segmented/Lt/slit_beam/whole"
save_dir = "/camin1/chlee/latent-diffusion-keratopathy/data"

# 파일 리스트 가져오기
slit_files = glob.glob(os.path.join(slit_dir, "*SLIT-1.png"))
slit_beam_files = glob.glob(os.path.join(slit_beam_dir, "*SLIT_BEAM-1.png"))

# key 만들기 (환자번호_날짜_Lt)
def get_key(filepath, slit=True):
    filename = os.path.basename(filepath)
    # slit: 17172038_20220530_Lt_SLIT-1.png
    # slit_beam: 17172038_20220530_Lt_SLIT_BEAM-1.png
    if slit:
        key = filename.replace('_SLIT-1.png', '')
    else:
        key = filename.replace('_SLIT_BEAM-1.png', '')
    return key

# slit dict: key->file, slit_beam dict: key->file
slit_dict = {get_key(f, slit=True): f for f in slit_files}
slit_beam_dict = {get_key(f, slit=False): f for f in slit_beam_files}

# 공통 key 추출
common_keys = sorted(list(set(slit_dict.keys()) & set(slit_beam_dict.keys())))

# split
train_keys, testval_keys = train_test_split(common_keys, test_size=0.4, random_state=42)
val_keys, test_keys = train_test_split(testval_keys, test_size=0.5, random_state=42)

splits = {
    "train": train_keys,
    "val": val_keys,
    "test": test_keys,
}

# 저장 함수
def write_paths(filelist, save_path):
    with open(save_path, "w") as f:
        for file in filelist:
            f.write(file + "\n")

for split in ["train", "val", "test"]:
    slit_split_files = [slit_dict[k] for k in splits[split]]
    slit_beam_split_files = [slit_beam_dict[k] for k in splits[split]]

    write_paths(slit_split_files, os.path.join(save_dir, f"L_slit_{split}_index.txt"))
    write_paths(slit_beam_split_files, os.path.join(save_dir, f"L_slit_beam_{split}_index.txt"))

print("파일 저장 완료!")


파일 저장 완료!
